In [2]:
import praw
import pandas as pd

In [4]:
# Create a read-only Reddit instance
reddit = praw.Reddit(client_id = '7xzWT1ks3sItJlMFs675sA',
                     client_secret = '9jshG6vAXfYXZv0zRCfiCgxOcA51pg',
                     redirect_url = 'http://localhost:8080',
                     user_agent = 'Whole_Ad194')